# [Pytorch-lightning](https://www.pytorchlightning.ai/)

Данная библиотека является оберткой над библиотекой `pytorch`. Она позволяет существенным образом сократить количество написанного кода (который часто повторяется, например циклы обучения, перенос данных и модели на GPU), облегичить запуск экспериментов, а также автоматически искать гиперпараметры моделей. 

На главной странице сайта можно увидеть беглое сравнение кода написанного на `pytorch` и на `pytorch-lightning`.

Далее сравним как изменится код написанный нами ранее на чистом `pytorch` с кодом написанным с использованием `pytorch-lighning` и убедимся в том, что данная утилита действительно позволяет избавиться от дублицирования одного и того же кода раз за разом.

В основе данной библиотеки лежат два класса `LightningModule` служащий для конфигурирования модели и её параметров; и `Trainer` служащий для запуска и контролирования обучения.

In [1]:
import comet_ml
import pytorch_lightning as pl
from pytorch_lightning.loggers import CometLogger

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms as T
import torchvision.transforms.functional as TF
from torchvision import utils

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

## LightningModule
Как уже было упомянуто, данный класс служит для созданния и конфигурирования обучаемой модели, рассмотрим его использование в нашем случае

In [3]:
class FashionMnistModel(pl.LightningModule):
    def __init__(self, model: nn.Module, loss):
        """
        :param model: непосредственно нейронная сеть
        :param loss: функция ошибки
        """
        super().__init__()
        self.model = model
        self.loss = loss

    def forward(self, x):
        """
        Метод аналогичный методу forward() в torch.nn.Module
        """
        out = self.model(x)
        return out
    
    def configure_optimizers(self):
        """
        Метод в котором можно конфигурировать оптимизатор
        """
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        # Также здесь можно настраивать и learning rate schedule 
        lr_scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.999)
        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}
    
    def training_step(self, train_batch, batch_idx):
        """
        Наш новый тренировочный цикл.
        Если посмотреть где вызывается данный метод, то под капотом в 
        pytorch_lightning написано ровно тоже самое, что мы писали вручную на pytorch:
        
        for batch_idx, batch in enumerate(train_dataloader):
            loss = training_step(batch, batch_idx)  # А вот и наш training_step!
            # clear gradients
            optimizer.zero_grad()
            # backward
            loss.backward()
            # update parameters
            optimizer.step()
        """
        img, label = train_batch
        preds = self.model(img)
        loss = self.loss(preds, label)
        self.log('train_loss', loss)
        return loss  # Важно не забыть вернуть loss

    def validation_step(self, val_batch, batch_idx):
        """
        Та часть тренировочного цикла, которая отвественна за валидацию.
        Т.к. мы хотим вычислять метрики по всему валидационному набору данных, 
        непосредственное вычисление метрик мы напишем в следующем методе.
        
        Данный метод вызывается для каждого батча. Именно поэтому здесь
        мы не можем вычислить метрики для всей эпохи (всего набора)
        """
        imgs, labels = val_batch
        preds = self.model(imgs)
        loss = self.loss(preds, labels)
        return {'loss': loss, 'preds': preds, 'labels': labels}
    
    def validation_epoch_end(self, validation_step_outputs):
        """
        validation_step_outputs - список возвращенных `validation_step` значений
        """
        all_losses = torch.hstack(list(map(lambda item: item['loss'], validation_step_outputs)))
        all_preds = torch.vstack(list(map(lambda item: item['preds'], validation_step_outputs)))
        all_labels = torch.hstack(list(map(lambda item: item['labels'], validation_step_outputs)))
        correct = (all_preds.argmax(dim=1) == all_labels).type(torch.float).sum().item()
        acc = correct / all_labels.shape[0]
        loss = torch.mean(all_losses)
        self.log_dict({'Accuracy': acc, "Val. Loss": loss}, prog_bar=True)

## Этап подготовки данных не изменился

In [4]:
augmentation = T.Compose([
    T.RandomRotation((-30, 30)),
    T.ToTensor(),
    T.Normalize((0.5,), (0.5,))
])

In [5]:
dataset = torchvision.datasets.FashionMNIST(
    root='./data', 
    download=False, 
    train=True,
    transform=augmentation
)

dataloader_train = torch.utils.data.DataLoader(
    dataset,
    batch_size=1024,
    shuffle=True,
    num_workers=4
)

dataloader_val = torch.utils.data.DataLoader(
    dataset,
    batch_size=1024,
    shuffle=False,
    num_workers=4,
)

In [6]:
class Resnet18(nn.Module):
    def __init__(self, n_channels: int = 1, n_classes: int = 10):
        super().__init__()
        self.resnet = torchvision.models.resnet18(pretrained=False)
        # Заменим первую свертку
        self.resnet.conv1 = nn.Conv2d(in_channels=n_channels, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
        self.resnet.fc = nn.Linear(in_features=self.resnet.fc.in_features, out_features=n_classes)
    
    def forward(self, x):
        hx = self.resnet(x)
        return hx

In [7]:
# Используем снова нашу Resnet18 для FashionMNIST
network = Resnet18()

## Создадим модель

In [8]:
model = FashionMnistModel(network, torch.nn.CrossEntropyLoss())

## Применим класс Trainer для начала обучения

У конструктора данного класса есть больше число разнообразных параметров, подробно они описаны в документации. Нас они сильно не интересует. Обратим лишь внимание на параметр `gpus`, если данный параметр `!= 0` и `!=None`, тогда вычисления будут проводиться на 'GPU'. Заметьте что больше нам не надо беспокоится о переносе данных на GPU вручную.

*Также есть параметры `devices` и `accelerator` которые также позволяют выбирать где будут выполняться вычисления.*

In [9]:
trainer = pl.Trainer(
    gpus=1, 
    max_epochs=10,
    check_val_every_n_epoch=1
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Обучение запускается вызовом метода `fit`

In [10]:
trainer.fit(model, dataloader_train, dataloader_val)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]

  | Name  | Type             | Params
-------------------------------------------
0 | model | Resnet18         | 11.2 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.701    Total estimated model params size (MB)


Epoch 0:  50%|█████     | 59/118 [00:05<00:05, 11.35it/s, loss=0.455, v_num=39]
Validating: 0it [00:00, ?it/s]
Epoch 1:  51%|█████     | 60/118 [00:05<00:04, 11.86it/s, loss=0.399, v_num=39, Accuracy=0.832, Val. Loss=0.460] 
Validating: 0it [00:00, ?it/s]
Epoch 2:  51%|█████     | 60/118 [00:05<00:04, 11.83it/s, loss=0.355, v_num=39, Accuracy=0.843, Val. Loss=0.420] 
Validating: 0it [00:00, ?it/s]
Epoch 3:  51%|█████     | 60/118 [00:05<00:04, 11.77it/s, loss=0.333, v_num=39, Accuracy=0.858, Val. Loss=0.382] 
Validating: 0it [00:00, ?it/s]
Epoch 4:  51%|█████     | 60/118 [00:05<00:04, 11.72it/s, loss=0.317, v_num=39, Accuracy=0.863, Val. Loss=0.364] 
Validating: 0it [00:00, ?it/s]
Epoch 5:  51%|█████     | 60/118 [00:05<00:04, 11.65it/s, loss=0.307, v_num=39, Accuracy=0.879, Val. Loss=0.320] 
Validating: 0it [00:00, ?it/s]
Epoch 6:  51%|█████     | 60/118 [00:05<00:04, 11.63it/s, loss=0.292, v_num=39, Accuracy=0.888, Val. Loss=0.298] 
Validating: 0it [00:00, ?it/s]
Epoch 7:  51%|█████

У `Trainer` также есть методы для проведения валидации и тестирования модели.

In [11]:
trainer.validate(model, dataloader_val)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]


Validating:  95%|█████████▍| 56/59 [00:02<00:00, 27.28it/s]--------------------------------------------------------------------------------
DATALOADER:0 VALIDATE RESULTS
{'Accuracy': 0.9046000242233276, 'Val. Loss': 0.253422349691391}
--------------------------------------------------------------------------------
Validating: 100%|██████████| 59/59 [00:02<00:00, 22.50it/s]


[{'Accuracy': 0.9046000242233276, 'Val. Loss': 0.253422349691391}]

# Выводы

Обратите внимание, на то, как `pytorch-lightning` организовал код и уменьшил необходимость дублицировать вновь и вновь один и тот же код. Это далеко не все возможности предоставляемые данной библиотекой. Она также позволяет легко переключать режим точности (FP32->FP16), добавлять логгирование (в том числе с помощью упомянутых comet.ml и т.д.), производить оптимизацию и профилирование нейронной сети и многое другое. Все возможности которыми обладает данная библиотека описаны в её докумментации. Также в документации разработчики добавили небольшие видео-ролики в которых коротко рассказывают о том или ином параметре/методе. 